# HSE-2024 text classification project

## Table of content
* [Imports](#c1)
* [Data loading and preprocessing](#c2)
* [Custom dataset](#c3)
* [Custom model](#c4)
* [Trainer](#c5)
* [Model training](#c6)

### Imports <a class="anchor" id="c1"></a>

In [1]:
import numpy as np
import pandas as pd

import torch
from sklearn.model_selection import train_test_split

import os
import re
import string

from classes import Preprocessor, MyModel, Trainer

### Data loading and preprocessing <a class="anchor" id="c2"></a>

In [2]:
RANDOM_STATE = 42
torch.manual_seed(RANDOM_STATE)

In [3]:
base_dir = 'data/'
if not os.path.exists(base_dir):  # создадим папку, куда будем сохранять модели, и где будут лежать данные
    os.makedirs(base_dir)

In [4]:
texts = pd.read_csv(f'{base_dir}texts_and_metadata.txt', sep='\t')
texts.sample(3)

,document.id,source,stage,source_text,lemm_text,city,region,date
8508,795951389,iqbuzz,3,"[[id5284111|Валентин], Вот даже в твоих словах...",валентин твой слово нотка тот равный кома высо...,Ноябрьск,Ямало-Ненецкий АО,2015-03-07
258,1032649600,iqbuzz,2,"[[id169546378|Лавъ-Изъ], Государство в коем пр...",государство кой преобладание один нация более ...,Прокопьевск,Кемеровская область,2015-09-21
7685,758915861,iqbuzz,3,Молодые и свободные. Их дружбу не купишь за бу...,легко молодая сезон серия молодая свободный др...,Москва,Москва,2015-02-12


In [5]:
data = pd.read_csv(f'{base_dir}coding_results.txt', low_memory=False, sep='\t')
data.sample(3)

,document.id,source,stage,data,assessor,seed_eth_group,for_questions_about_text,do_text_make_sense_raw,do_text_make_sense_recoded,has_ethnonym_raw,...,represent_ethicity_raw,represent_ethicity_meaning,is_ethicity_superior_raw,is_ethicity_superior_meaning,is_ethicity_aggressor_raw,is_ethicity_aggressor_meaning,is_ethicity_dangerous_raw,is_ethicity_dangerous_meaning,comment,old_id
16915,722771149,iqbuzz,3,2017-03-28 13:11:59,Tatiana,башкир,0,yes,1,several,...,1.0,no,3.0,irrel,3.0,irrel,1.0,no,"Казахи, Уйгуры, Дунгане, Хакасы, Татары, Таджики",722771149
49881,1068054308,iqbuzz,3,2017-03-28 21:28:24,adzhigitova,абхаз,1,yes,1,several,...,2.0,unk,3.0,irrel,2.0,agressor,2.0,yes,NaN,1068054308
77273,1064714418,iqbuzz,2,2016-10-14 16:52:28,Tankly,удмурт,0,yes,1,several,...,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,удмурт_10


In [6]:
df = texts.merge(data, how='left')
df.sample(5)

,document.id,source,stage,source_text,lemm_text,city,region,date,data,assessor,...,represent_ethicity_raw,represent_ethicity_meaning,is_ethicity_superior_raw,is_ethicity_superior_meaning,is_ethicity_aggressor_raw,is_ethicity_aggressor_meaning,is_ethicity_dangerous_raw,is_ethicity_dangerous_meaning,comment,old_id
30991,1069099174,iqbuzz,2,"[[id328761257|Ашотик], да не. Помнишь я тебе к...",ашотик помнить кидать вт немного помогать один...,Иркутск,Иркутская область,2015-10-24,2016-10-19 00:02:58,dianasadr,...,1.0,no,1.0,low,3.0,irrel,1.0,no,NaN,чурка_4
7096,749646192,iqbuzz,2,"[[id213010462|Иван], вот и всплыла твоя зомбир...",иван всплыть твой ваш русский исключительность...,Петрозаводск,Карелия,2015-02-08,2016-10-18 23:52:52,skuchilina,...,1.0,no,3.0,irrel,3.0,irrel,1.0,no,Негр - афроамериканец,вепс_58
43960,730222073,iqbuzz,3,Ну-да и воевать до последнего украинца или рус...,воевать последний украинец руский уйти воевать...,Нефтеюганск,Ханты-Мансийский Автономный окру,2015-01-28,2017-03-25 18:45:56,skuchilina,...,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,730222073
37248,535811168,iqbuzz,3,"аэропорт, известная история, но не безнаказанн...",re сочинский олимпиада конец сказка аэропорт и...,Коломна,Московская область,2014-02-12,2017-03-24 21:26:20,skuchilina,...,2.0,unk,3.0,irrel,3.0,irrel,1.0,no,NaN,535811168
54939,818208150,iqbuzz,3,"\\""Я чуваш. Есть такая русская национальность\...",re гостиный чуваш такой русский национальность...,Приозерск,Ленинградская область,2015-03-19,2017-03-31 21:06:31,Tatiana,...,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,818208150


In [7]:
df.shape

(84784, 61)

In [8]:
df['do_text_make_sense_raw'].value_counts()

do_text_make_sense_raw
yes     78372
no       4334
lang     1890
joke      188
Name: count, dtype: int64

In [9]:
df.drop(df[df['do_text_make_sense_raw'] == 'no'].index, inplace=True)

In [10]:
ops = ['is_text_positive_recoded', 'is_text_neg_recoded']
df[ops] = df[ops].apply(pd.to_numeric, errors='coerce')
pos_ = df['is_text_positive_recoded']
neg_ = df['is_text_neg_recoded']
df.loc[(pos_ > 0) & (neg_ < 0), ops] = None
df.loc[neg_ < 0, 'text_sentiment'] = -1
df.loc[pos_ > 0, 'text_sentiment'] = 1
df.loc[(neg_ == 0) & (pos_ == 0), 'text_sentiment'] = 0

In [11]:
args = ['text_sentiment', 'has_eth_conflict_raw', 'has_pos_eth_interaction_raw', 'opinion_about_ethnonym_recoded',
        'is_ethicity_superior_meaning', 'is_ethicity_aggressor_meaning',
        'is_ethicity_dangerous_meaning']
MAX_SPOIL = len(args)
topic_to_russian = {'culture': 'культура', 'economics': 'экономика', 'ethicity': 'этничность', 'history': 'история',
                    'humour': 'юмор', 'daily_routine': 'рутина', 'migration': 'миграция', 'other': 'другая',
                    'politics': 'политика', 'religion': 'религия', 'society_social': 'социальная'}
MAX_SPOIL

7

In [12]:
df['has_pos_eth_interaction_raw'].value_counts()

has_pos_eth_interaction_raw
no     62783
yes    13603
unk     1986
Name: count, dtype: int64

In [13]:
var_vocab = {
    'text_sentiment': {'labels': {-1.0: 'этот текст является негативным', 0: 'этот текст является нейтральным',
                                  1.0: 'этот текст является позитивным'}, 'aspect_level': False, 'prompt': 'тональность текста'},
    'has_eth_conflict_raw': {
        'labels': {'yes': 'в тексте есть этнический конфликт', 'no': 'в тексте этнический конфликт отсутствует',
                   'unk': None},
        'aspect_level': False, 'prompt': 'оцени наличие этнического конфликта'},
    'has_pos_eth_interaction_raw': {'labels': {'yes': 'в тексте есть позитивное взаимодействие между этичностями',
                                               'no': 'в тексте не зафиксировано позитивного взаимодействия этичностей',
                                               'unk': None},
                                    'aspect_level': False, 'prompt': 'наличие позитивного взаимодействия этничностей'},
    'opinion_about_ethnonym_recoded': {'labels': {-1: 'мнение об этничности \'{}\' отрицательное',
                                                  0: 'мнение об этничности \'{}\' нейтральное или не зафиксировано',
                                                  1: 'мнение об этничности \'{}\' положительное'},
                                       'aspect_level': True, 'prompt': 'мнение о {} в тексте'},
    # 'represent_ethicity_meaning': {'labels': {}, 'aspect_level': True}, # not using yet
    'is_ethicity_superior_meaning': {'labels': {'high': 'этничность \'{}\' является доминирующей',
                                                'low': 'этничность \'{}\' является отчасти доминирующей',
                                                'irrel': None},
                                     'aspect_level': True, 'prompt': 'является ли этничность {} доминирующей'},
    'is_ethicity_aggressor_meaning': {'labels': {'agressor':
                                                     'этничность \'{}\' является агрессором',
                                                 'victim': 'этничность \'{}\' является жертвой',
                                                 'irrel': None},
                                      'aspect_level': True, 'prompt': 'является ли этничность {} агрессором'},
    'is_ethicity_dangerous_meaning': {
        'labels': {'yes': 'этничность \'{}\' является опасной',
                   'no': 'этничность \'{}\' не является опасной',
                   'irrel': None},
        'aspect_level': True, 'prompt': 'является ли этничность {} опасной'},
}

In [14]:
df = df.fillna(np.nan).replace([np.nan], [None])

In [15]:
ids = df['document.id'].unique()
ids.shape  # 14196 после дропа по do_text_make_sence = no

(14196,)

In [16]:
def clean(text):
    text = text.apply(lambda x: str(x))
    CLEANR = [re.compile('<.*?>'), re.compile("\[.*?\]")]
    for i in CLEANR:
        text = text.apply(lambda x: re.sub(i, '', x))
    text = text.apply(lambda x: x.replace('\\', ''))
    text = text.apply(lambda x: re.sub(r"([" + re.escape(string.punctuation) + r"])\1+", r"\1", x))
    text = text.apply(lambda x: re.sub(r"http\S+", '', x))
    text = text.apply(lambda x: re.sub(r"\r", '', x))
    text = text.apply(lambda x: re.sub(r'\s+', ' ', x))
    text = text.apply(lambda x: re.sub(r'\s+', ' ', x))
    text = text.apply(lambda x: re.sub('"+','"', x))
    text = text.apply(lambda x: re.sub("'+","'", x))
    items = string.punctuation + " "
    text = text.apply(lambda x: x.lstrip(items) if isinstance(x, str) else x)
    return text

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:3: SyntaxWarning: invalid escape sequence '\['
C:\Users\bentodan\AppData\Local\Temp\ipykernel_432\3087058704.py:3: SyntaxWarning: invalid escape sequence '\['
  CLEANR = [re.compile('<.*?>'), re.compile("\[.*?\]")]


In [17]:
df['source_text'] = clean(df['source_text'])

In [18]:
df['source_text'][0]

'но у вас же бред написан. Какими русскими? Вообще то там грузины воевали с Абхазами. Это исторический факт. А статья может быть на 10% правдива. Почему вы верите, какой-то статье?'

In [19]:
preprocessor = Preprocessor.Preprocessor(df=df, args=args, var_vocab=var_vocab, topic_to_russian=topic_to_russian)

In [20]:
id_ = df['document.id'].sample().values[0]
descr, text = preprocessor.fit(id_)
print(id_, descr, text, sep='\n')

783920705
этот текст является позитивным, в тексте этнический конфликт отсутствует, в тексте не зафиксировано позитивного взаимодействия этичностей, мнение об этничности 'армянин' положительное, этничность 'армянин' не является опасной

5 марта в 19:00 Областной дом народного творчества приглашает всех на долгожданное событие – областной праздник-конкурс национальных красавиц Дона «Краса Юга России»! Праздник национальных культур и красоты проводится второй раз на территории Ростовской области. В празднике-конкурсе примут участие лучшие представительницы народов, проживающих на Дону. Армянскую диаспору Дона представит Петросян Лиана.Ей наша поддержка очень нужна. Давайте все вместе соберемся и пойдем болеть за нашу представительницу. Билеты стоят 300 руб


In [21]:
descr_spoiled, text = preprocessor.fit(id_, spoil_size=len(preprocessor.args))
print(descr, descr_spoiled, sep='\n')

этот текст является позитивным, в тексте этнический конфликт отсутствует, в тексте не зафиксировано позитивного взаимодействия этичностей, мнение об этничности 'армянин' положительное, этничность 'армянин' не является опасной

этот текст является негативным, мнение об этничности 'армянин' отрицательное, этничность 'армянин' является доминирующей, этничность 'армянин' является жертвой



In [22]:
descr_topics, _ = preprocessor.fit(id_, topic=True)
descr_topics_spoiled, _ = preprocessor.fit(id_, topic=True, topic_spoil=1)
print(descr_topics, descr_topics_spoiled, sep='\n')

Текст имеет темы: культура, этничность, этот текст является позитивным, в тексте этнический конфликт отсутствует, в тексте не зафиксировано позитивного взаимодействия этичностей, мнение об этничности 'армянин' положительное, этничность 'армянин' не является опасной

Текст имеет темы: экономика, история, этот текст является позитивным, в тексте этнический конфликт отсутствует, в тексте не зафиксировано позитивного взаимодействия этичностей, мнение об этничности 'армянин' положительное, этничность 'армянин' не является опасной



### Custom dataset <a class="anchor" id="c3"></a>

In [23]:
process_ids, test_ids = train_test_split(ids, test_size=0.2, random_state=RANDOM_STATE)
train_ids, validate_ids = train_test_split(process_ids, train_size=0.75, random_state=RANDOM_STATE)

train = df.loc[df['document.id'].isin(train_ids)]
test = df.loc[df['document.id'].isin(test_ids)]
validate = df.loc[df['document.id'].isin(validate_ids)]
train.shape, test.shape, validate.shape  # percents are ≈ (60%, 20%, 20%)

((47999, 62), (16170, 62), (16281, 62))

### Custom model <a class="anchor" id="c4"></a>

In [24]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device used: {device}.")

Device used: cpu.


In [25]:
model = MyModel.MyModel(device) # turn on the developer mode here
print(f"Model loaded. Model tokenizer is {model.tokenizer}.")

Model loaded. Model tokenizer is BertTokenizerFast(name_or_path='cointegrated/rubert-tiny2', vocab_size=83828, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}.


In [26]:
model.reinitialize()
print(f"Model reloaded. Model tokenizer is {model.tokenizer}.")

Model reloaded. Model tokenizer is BertTokenizerFast(name_or_path='cointegrated/rubert-tiny2', vocab_size=83828, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}.


### Trainer <a class="anchor" id="c5"></a>

In [27]:
params = {
    'batch_size': [8, 16],
    'lr': [1e-5, 1e-6, 1e-4],
    'max_spoil': range(1, MAX_SPOIL + 1, 2),
    'spoil_proba': np.arange(0.2,  0.8 + 0.1, 0.1)
}

In [28]:
trainer = Trainer.Trainer(MyModel.MyModel, device, train, validate, test, preprocessor=preprocessor, params=params)

### Model training <a class="anchor" id="c6"></a>

In [29]:
torch.cuda.empty_cache()  # just in case

In [2]:
# trainer.choose_model() # uncomment if you want to find best model

In [3]:
# trainer.save()

In [ ]:
# trainer.plot_loss(on_train=False)

In [ ]:
# trainer.plot_metrics(on_train=False)

### Генерация описаний

In [39]:
from gpt_classes import GPTDataset, GPTModel, GPTTrainer

In [40]:
gptdataset = GPTDataset.GPTDataset(df, preprocessor=preprocessor, args=var_vocab)

In [41]:
print(gptdataset[0][0])

Задание: Сгенерируй описание следующего текста, оцени тональность текста, оцени наличие этнического конфликта, наличие позитивного взаимодействия этничностей, мнение о абхаз в тексте, является ли этничность абхаз доминирующей, является ли этничность абхаз агрессором, является ли этничность абхаз опасной.
Текст: но у вас же бред написан. Какими русскими? Вообще то там грузины воевали с Абхазами. Это исторический факт. А статья может быть на 10% правдива. Почему вы верите, какой-то статье?
Описание: этот текст является негативным, в тексте есть этнический конфликт, в тексте не зафиксировано позитивного взаимодействия этичностей



In [42]:
from torch.utils.data import DataLoader

In [43]:
max_items = 100 # for now on
gptTrainer = GPTTrainer.GPTTrainer(GPTModel.GPTModel, device, train[:max_items], validate[:max_items], test[:max_items], preprocessor=preprocessor, dataset=GPTDataset.GPTDataset, args=var_vocab, dataloader=DataLoader)
# gptTrainer = GPTTrainer.GPTTrainer(GPTModel.GPTModel, device, train, validate, test, preprocessor=preprocessor, dataset=GPTDataset.GPTDataset, args=var_vocab, dataloader=DataLoader)

In [45]:
print(gptTrainer.model.my_generate('Что такое питон?')) # слон - это змея:)

Что такое питон?
Питон - это млекопитающее, которое питается плодами растений и животных.

Какие у вас ассоциации со словом "питон"?
Птица
птица, которая летает, летающая, летящая
Это птица. Птица - птица, которую можно увидеть в небе, а можно и не увидеть. Это птица-попугай, который живет на земле. А еще это птица - птичка, которой можно летать. И еще - птеродактиль


In [ ]:
gptTrainer.train(1)

In [ ]:
print(gptTrainer.model.my_generate('Что такое питон?'))

In [ ]:
for id_, generated in gptTrainer.data[0][0]['generated']:
    print(id_)
    print(generated)
    print("\n-----\n")
    break

In [ ]:
print(gptTrainer.data[0][0]['matched_with_descr'])

In [ ]:
GPTparams = {
    'cut_data': [True, False],
    'batch_size': [10, 20],
    'lr': [1e-5, 1e-6, 1e-4],
    'ngrams': [1, 2, 3, 4, 5]
}

In [ ]:
gptTrainer.choose_model(GPTparams)

### Add f1_score 

In [46]:
sample_num = 5
ids = df['document.id'].sample(sample_num).tolist()

In [47]:
ids[:3]

[552763, 891302411, 927506040]

In [61]:
values = dict()
labels = dict()

In [62]:
for var, val in var_vocab.items():
    values[var] = dict()
    labels[var] = dict()
    val = val['labels']
    for id_ in ids:
        data = df.loc[df['document.id'] == id_]
        values[var][id_] = None
        labels[var][id_] = res
        if not var_vocab[var]['aspect_level']:
            data = data.drop_duplicates(subset='assessor')[var]
            res = preprocessor.define(data)
            if res is not None:
                values[var][id_] = val[res]

In [50]:
model_preds = dict()

In [51]:
f1_df = df[df['document.id'].isin(ids)]
f1_df.shape

(32, 62)

In [53]:
dir_ = "data" # fill with correct path to dir

model = GPTModel.GPTModel(device)

for filename in os.listdir(dir_):
    
    f = os.path.join(dir_, filename)
    if f.startswith(dir_ + "\\epoch"):
        print(f"start evaluating {f}")

        model_preds[f] = dict()
        model.load_state_dict(torch.load(f))
        model.eval()

        for var, val in var_vocab.items():
            if var_vocab[var]['aspect_level']:
                continue
            print(var)
            model_preds[f][var] = dict()
            prompt = val['prompt']
            for id_ in ids:
                text = f1_df[f1_df['document.id'] == id_].iloc[0].lemm_text
                data = f'Задание: {prompt or ""}\nТекст: {text or ""}\nОписание:'
                generated = model.my_generate(data)
                model_preds[f][var][id_] = generated
        break

start evaluating data\epoch_0_num_0
text_sentiment
has_eth_conflict_raw
has_pos_eth_interaction_raw
opinion_about_ethnonym_recoded
is_ethicity_superior_meaning
is_ethicity_aggressor_meaning
is_ethicity_dangerous_meaning


In [54]:
from sklearn.metrics import f1_score

In [55]:
results = dict()

In [71]:
for filename in os.listdir(dir_):
    f = os.path.join(dir_, filename)
    if f.startswith(dir_ + "\\epoch"):
        results[f] = dict()
        for var in var_vocab:
            if var_vocab[var]['aspect_level']:
                continue
            vals = values[var]
            model_vals = model_preds[f][var]
            classes = []
            preds = []
            for id_ in ids:
                print(model_vals[id_], vals[id_], sep = "\n---\n", end = "\n====\n")
                preds.append(vals[id_] in model_vals[id_] if model_vals[id_] and vals[id_] else False)
                classes.append(labels[var][id_])
            # f1 = f1_score(preds, classes)
            # results[f][var] = f1
            print(*classes)
            print(*preds, end="\n====\n")
            break

        break

Задание: тональность текста
Текст: кавказец онподнимаеттрубку среда приходить центр город убивать отец хороший минутспустя звонить среда неприедешь намне парень отвечать хороший черезнесколько час емузвонят егодрузья братан передрягупопали приезжать центр средув хороший онспросил усвоей материть емуделать вещий егожизни убийцаотца будущий жена друг мать отвечать отец невернешь девушка встречать езжать кдрузьям дружба святой средув придти высылать машин открывать заднююдверь сидеть егодевушка всвадебном платье открываютбагажник убийца отец окружать
Описание:
1.\t

2. \t\thttps://www.youtube.com/watch?v=z-Z-zZZ_Z
3.  
4. 
5.
6. 

7.   
8.    
9.        
10.                
11. https://t.co/zzYzWzJZzQ
12. http://vk.cc/2QQ
---
None
====
Задание: тональность текста
Текст: видео нужно крутить канал вместо реклама дмитрий фашист русский белорус украинец казах грузин другой народ ссср одинакий русский русский народ прекрасно помнить горелый хатынь залить кровь бабий ярый пылать маленькая украи